왜 Reranker를 쓰나요? (주요 장점)
- 정확도 향상: 단순 벡터 검색은 단어의 의미가 비슷하면 관련 없는 내용도 가져올 수 있는데, Reranker가 이를 걸러줍니다.

- LLM 비용 절감: 연관성이 높은 상위 3~5개 문서만 LLM에 전달하므로, 불필요한 토큰 낭비를 줄이고 답변의 질을 높입니다.

- Context Window 최적화: LLM이 한 번에 읽을 수 있는 양은 제한되어 있으므로, "진짜 핵심"만 전달하는 것이 유리합니다.

## 학습 방법
### 1. 포인트와이즈(Pointwise)
: 개별 쿼리-문서 쌍의 관련성 점수 예측
- 한 명씩 면접장에 들어오게 해서 절대평가로 점수를 매기는 것
### 2. 페어와이즈(Pairwise)
: 두 문서 간의 상대적 관련성 비교
- 두 명을 동시에 불러서 토너먼트식 비교
### 3. 리스트와이즈(Listwise)
: 후보군에 있는 모든 문서(List)를 한꺼번에 모델에 넣고, 전체적인 최적의 순서를 한 번에 결정
- 지원자 10명을 한 방에 넣고 1등부터 10등까지 전체 순위를 한 번에 매기는 것.

In [ ]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
!pip install -qU langchain langchain-community langchain-text-splitters sentence-transformers langchain_huggingface faiss-cpu

In [ ]:
pip install -r https://raw.githubusercontent.com/teddylee777/langchain-kr/main/requirements.txt

# 4가지 Reranker 라이브러리

## 01. Cross Encoder Reranker

가장 근본적인 형태입니다. SentenceTransformers 라이브러리를 통해 직접 모델을 로드

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 설정
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

# 문서로부터 FAISS 인덱스 생성 및 검색기 설정
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 10}
)

In [ ]:
# 질의 설정
query = "Word2Vec에 대해서 알려줄래?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

# 결과 문서 출력
pretty_print_docs(docs)

Word2Vec은 Word Embedding의 한 종류이기에 결과가 이렇게 나온거임
-> reranker 필요

ContextualCompressionRetriever ( Retriever + Reranker )

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("Word2Vec 에 대해서 알려줄래?")

# 문서 출력
pretty_print_docs(compressed_docs)


## 02. Cohere Reranker

유료 API 서비스입니다. 모델 내부를 볼 수는 없지만

In [ ]:
!pip install -qU cohere

Cohere API 키 발급 안함. 그리고 위 코드에서 아래쪽만 다름

In [ ]:
# 검색기 초기화
retriever = FAISS.from_documents(
    texts, CohereEmbeddings(model="embed-multilingual-v3.0")
).as_retriever(search_kwargs={"k": 10})

# 문서 검색
docs = retriever.invoke(query)

# 문서 출력
pretty_print_docs(docs)

> Cohere의 Retriever 성능이 워낙 좋아서 리랭커 없이도 충분한 품질이 나옴

reranker

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# 문서 재정렬 모델 설정
compressor = CohereRerank(model="rerank-multilingual-v3.0")

# 문맥 압축 검색기 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("Word2Vec 에 대해서 알려줘!")

# 압축된 문서 출력
pretty_print_docs(compressed_docs)


## 03. Jina Reranker

긴 문장(8k 토큰 이상)을 처리하는 데 특화

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 검색기 초기화
retriever = FAISS.from_documents(
    texts, OpenAIEmbeddings()
).as_retriever(search_kwargs={"k": 10})

# 질의문
query = "Word2Vec 에 대해서 설명해줘."

# 문서 검색
docs = retriever.invoke(query)

# 문서 출력
pretty_print_docs(docs)

reranker는 단독으로 동작하는 것이 아니라

먼저 OpenAIEmbeddings 같은 모델로 후보군을 대충 추려낸 뒤(Retriever 단계), 그 후보들만 Jina Reranker에게 전달해 정밀 검사를 시키는 구조라서

위 코드는 Jina Reranker를 적용하기 **'전'**의 상태를 보여줘 성능 차이를 보여주려는 의도!

In [ ]:
from ast import mod
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank

# JinaRerank 압축기 초기화
compressor = JinaRerank(model="jina-reranker-v2-base-multilingual", top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 관련 문서 검색 및 압축
compressed_docs = compression_retriever.invoke(
    "Word2Vec 에 대해서 설명해줘."
)
pretty_print_docs(compressed_docs)

## 04. FlashRank Reranker

성능보다 **'속도'**에 올인한 초경량 및 초고속 Python 라이브러리

In [ ]:
!pip install -qU flashrank

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 각 텍스트에 고유 ID 추가
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

# 검색기 초기화
retriever = FAISS.from_documents(
    texts, OpenAIEmbeddings()
).as_retriever(search_kwargs={"k": 10})

# 질의문
query = "Word2Vec 에 대해서 설명해줘."

# 문서 검색
docs = retriever.invoke(query)

# 문서 출력
pretty_print_docs(docs)


### Q. 다른 reranker 설명 문서에서의 코드와 달리 각 텍스트에 고유 ID를 추가하는 이유가 뭘까?

다른 Reranker들은 서버(cloud)형 서비스로, 문서를 보내면 서버 내부에서 알아서 인덱싱하고 처리한 뒤 결과를 돌려주니에 매핑 과정을 사용자가 신경 쓸 필요가 적지만

flash는 local에서 직접 돌아가다보니 내부적으로 문서들을 빠르게 구분하고 정렬하기 위해 **"숫자나 문자로 된 이름표(ID)"**가 반드시 있어야만 작동하도록 설계되어있어 그렇게 한다.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(temperature=0)

# 문서 압축기 초기화
compressor = FlashrankRerank(model="ms-marco-MultiBERT-L-12")

# 문맥 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke(
    "Word2Vec 에 대해서 설명해줘."
)

# 문서 ID 출력
print([doc.metadata["id"] for doc in compressed_docs])

FlashrankRerank가 재정렬한 결과, 질문에 가장 적합하다고 판단된 문서들의 원래 '이름표(ID)'가 출력됨

In [ ]:
pretty_print_docs(compressed_docs)